**Задание по проекту.** 
Для его выполнения вам понадобится собранная коллекция документов и функция, составляющая обратный индекс по словам в коллекции.

Напишите функцию (или несколько отдельных логичный функций), которая по запросу $Q = q_1,..., g_n$ и коллекции $D$ сортирует выдачу подходящих документов. Будем считать документ подходящим, если он содержит хотя бы одно слово из запроса (из которого удалены стоп-слова). В качестве метрики используйте *Okapi BM25*.

Для проверки работы функции на вашем корпусе используйте запрос **каникулы на новый год и рождество**. Выведите ссылки в ipynb на первые десять докуменов в отсортированной выдаче(как во втором семинаре с помощью IPython.display) и их оценку BM25. Напомню, что ссылки на документы хрянятся в самих доках под тэгом @url.

Про что не забыть:
1. Лемматизируем запрос, удаляем стоп-слова => запрос готов
2. Лемматизируем слова в документах => документы готовы к подсчетам статистик по запросу

In [1]:
from collections import defaultdict
import nltk
import os
import re
import json
from pymystem3 import Mystem
from nltk import word_tokenize
import string
from nltk.corpus import stopwords
from IPython.display import HTML, display

In [2]:
def reversed_index(list_of_lemmas):
    reverse = defaultdict(list)
    for key,value in list_of_lemmas.items():
        for lemma in value:
            reverse[lemma].append(key)
    return reverse

In [3]:
def lemmatize_me(text):
    m = Mystem()
    lemma = m.lemmatize(text)

    #let's delete punctuation symbols
    lemma = [i for i in lemma if ( i not in string.punctuation )]

    #deleting stop_words
    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '–', 'к', 'на', '\n', '\t', ' '])
    lemma = [i.strip() for i in lemma if ( i not in stop_words )]

    #cleaning words
    #tokens = [i.replace("«", "").replace("»", "") for i in tokens]

    return lemma

In [10]:
list_of_lemmas = defaultdict(list)
length = 0

for root, dirs, files in os.walk('./text'):
    for i in files:
        if i[-4:] == '.txt':
            file = open(root + '/' + i, 'r', encoding='utf-8')
            file_name = i
            f = file.read()
            content = f.split("html",1)[1] 
            length += len(content.split())
            list_of_lemmas[file_name]=lemmatize_me(content)
file.close()

#json.dump(reversed_index(list_of_lemmas), open('reversed_index.json', 'w', encoding='utf-8'))
#avdl = length/24

289.2083333333333

In [4]:
json_data=open('reversed_index.json', 'r', encoding='utf-8')
j = json_data.read()
data = json.loads(j)
    
 # N- общее количество документов в коллекции, а  n(qi)— количество документов, содержащих  qi
# dl длина конкретного текста
# avdl средняя длина по всему корпусу
# qf частота слова в документе

In [5]:
def compute_tf(word, text):
    tf_text = 0
    for i in text:
        if i == word:
            tf_text += 1
    tf_text = tf_text/float(len(text))
    return tf_text

In [6]:
from math import log

N = 24
avdl = 2791.125
k1 = 2.0
b = 0.75

def score_BM25(n, qf, N, dl, avdl):
    K = compute_K(dl, avdl)
    IDF = log((N - n + 0.5) / (n + 0.5))
    frac = ((k1 + 1) * qf) / (K + qf)
    return IDF * frac
#N=24, avdl=2791.125, k1=2.0, b=0,75

def compute_K(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)))

In [8]:
def search(query, data):
    lem_query = lemmatize_me(query)
    result = defaultdict(set)
    for key, value in data.items():
        for word in lem_query:
            if word == key:
                result[word] = set(value)
    return result

query = 'из Москвы в Салехард'
#print(search(query,data))

def extract_info(text, key):
    url = re.search('@url (.+)', text).group(1)
    title = re.search('@ti (.+)', text).group(1)
    content = text.split("html",1)[1] 
    dl = len(content.split())
    qf = compute_tf(key, lemmatize_me(content))
    return url, title, content, dl, qf

def metrics(result, N=24, avdl=289.2083333333333, k1=2.0, b=0.75):
    output = {}
    for key, value in result.items():
        n = len(value)
        for file in value:
            f = open('./text/' + file, 'r', encoding='utf-8')
            text = f.read()
            url, title, content, dl, qf = extract_info(text, key)
            metric = score_BM25(n, qf, N, dl, avdl)
            link = '<a href="' + url + '">' + title + '</a>'
            if file not in output:
                output[link] = metric
            else:
                output[link] += metric
            f.close()
            
    for key in sorted(output, key=output.get, reverse=True):
        link = '{} — {}'.format(key, output[key])
        print(display(HTML(str(link))))
     #   print(key,value)
metrics(search(query,data)) 

None


None


None


None


None


...

None


None


None


None


None
